In [1]:
import sys
sys.path.append('/content')
from src.fluvius import USGS_Water_DB

%load_ext autoreload
%autoreload 2

db = USGS_Water_DB()
db.get_station_df()
db.station_df



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/root/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache


,site_no,site_name,Latitude,Longitude,geometry
0,01472157,"French Creek near Phoenixville, PA",40.1515,-75.6013,POINT (-75.60130 40.15150)
1,01478245,"White Clay Creek near Strickersville, PA",39.7475,-75.7708,POINT (-75.77080 39.74750)
2,01480617,"West Branch Brandywine Creek at Modena, PA",39.9618,-75.8013,POINT (-75.80130 39.96180)
3,01480870,East Branch Brandywine Creek below Downingtown...,39.9687,-75.6733,POINT (-75.67330 39.96870)
4,01481000,"Brandywine Creek at Chadds Ford, PA",39.8698,-75.5933,POINT (-75.59330 39.86980)
...,...,...,...,...,...
160,393806095273700,"Atchison County Lake near Horton, KS",39.6350,-95.4603,POINT (-95.46030 39.63500)
161,393817095260100,"Clear Creek at Decator Road near Horton, KS",39.6381,-95.4336,POINT (-95.43360 39.63810)
162,394126096073500,Black Vermillion River Tributary above Central...,39.6906,-96.1264,POINT (-96.12640 39.69060)
163,394146096085500,"Centralia Lake near Centralia, KS",39.6961,-96.1486,POINT (-96.14860 39.69610)


In [27]:
class USGS_Station_Data:
    def __init__(self, site_no, verbose=False):
        self.driver = create_driver()
        self.verbose = verbose
        self.site_no = site_no
        self.nrtwq_url = 'https://nrtwq.usgs.gov/explore/datatable'
        
    def get_url_text(self, url):
        driver.get(url)
        result = requests.get(url, allow_redirects=False)
        if result.status_code==200:
            if verbose:
                print('Data found!')
            soup = bs(result.text, 'html.parser') 
            return soup
        else:
            if verbose:
                print('Data does not exist')
            return None
        
    def process_soup(self, soup):
        data_raw = str(soup).split('\n')
        data_raw = [elem for elem in data_raw if not ('#' in elem)]
        data_split = [d.split('\t') for d in data_raw]
        y = (i for i,v in enumerate(data_split) if ('' in v))
        stop = next(y) #identify end of the string of continuous data
        cols = data_split[0]
        units = data_split[1]
        columns = [f'{c} ({u})' if ' ' not in u else f'{c}' for c,u in zip(cols,units) ]
        data = data_split[2:stop]
        df = pd.DataFrame(data=data, columns=columns)
        return df

    def get_water_url(self, attribute, year):
        pcode_list = {'discharge':'00060',\
              'turbidity':'63680',\
              'temperature':'00010',\
              'dissolved_oxygen':'00300',\
              'ssd':'99409'}
        timestep = 'uv'
        period = f'{year}_all'
        l = {'url_header':self.nrtwq_url, 'site_no':self.site_no, 'timestep':timestep}
        l['period'] = period
        l['pcode'] = pcode_list[attribute]
        url = f"{l['url_header']}?site_no={l['site_no']}&pcode={l['pcode']}&period={l['period']}&timestep={l['timestep']}&format=rdb&is_verbose=y"
        return url
    
    def get_station_attribute(self, attribute, year):
        water_url = get_water_url(self.site_no, attribute, year)
        textsoup = get_url_text(self.driver, water_url, verbose=False)
        out = None
        if textsoup is not None:
            out = process_soup(textsoup)
        return out 



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [/root/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache
